### imports

In [13]:
import instructor
from openai import OpenAI
import os
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

True

### load instructor and openai

In [10]:
# Patch the OpenAI client with Instructor
client = instructor.from_openai(OpenAI(api_key=os.environ['OPENAI_API_KEY']))

### user query/proposal idea 💡

In [1]:
user_proposal = input("Please enter the ai app you'd like to develop and the problem you are trying to solve:")

In [2]:
print(user_proposal)

I want to create a portfolio chatbot that answers questions about my experience and qualifications for a role that I might be applying to


### LLM #1 - Define Scope & Industry 🏭🗂️

### business opportunity w/ scope

### LLM #2 - very critical (pass ✅ or fail ❌ w/ explanation) 

### LLM #3 - scope 🛝 & find existing workflows 🔍📇

#### existing tools and workflows 🔍📇

In [ ]:
RAG = """
Overview of Retrieval-Augmented Generation (RAG)

RAG is a design pattern based on the paper Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks from 2020. It is not a framework, product, or product category. RAG is particularly used in the development of AI applications, especially “Question-Answer” based applications (ChatBots). Over the past year, it has gained popularity due to the increasing demand for LLM-based applications. The primary goal of RAG is to provide more relevant answers based on private data, reducing hallucinations and ensuring that answers are current. By retrieving relevant information and providing it to the LLM to compose an answer, RAG improves efficiency and is scalable to use other knowledge sources.

Growth and Integration of RAG Components
With the growth in demand, both big tech companies and startups have been developing components for the RAG pipeline. These are often heterogeneous systems that allow integration with other vendors' components. For instance, LangChain (an orchestration tool) can be used with Vertex AI (Google’s AI framework). Numerous variants of the RAG pipeline exist, and we will explore a few below.

Naive RAG
This basic variant serves as a starting point for RAG implementation. The process begins by ingesting and chunking data. This is analogous to tearing pages out of an encyclopedia and then embedding these chunks into a vector database. Various databases, such as MongoDB, Pinecone, Weaviate, and Qdrant, offer these capabilities. The RAG pipeline retrieves relevant chunks of data using an embedding model and passes this information to the LLM, which then composes an answer.

Key Steps:
	1.	Data Ingestion and Chunking: Breaking down data into manageable pieces.
	2.	Embedding: Converting chunks into numeric and storing in a vector database.
	3.	Retrieval: Using an embedding model to fetch relevant data chunks.
	4.	LLM Composition: LLM receives the retrieved data and generates the response.

Configuration and Selection Considerations
When setting up a RAG pipeline, various components and settings need to be considered:
	•	Text Ingest Tool
	•	Chunking Strategy
	•	Foundation Model
	•	Embedding Model
	•	Model Fine Tuning
	•	Vector Database and its Metadata
	•	Retrieval Methods
	•	Prompt Template
	•	Orchestration Tools
	•	Evaluation (Eval) Tools
	•	LLMOps
	•	UI Framework
Component Candidates:
	•	LLM: OpenAI, Anthropic, MetaAI, Mistral
	•	Embedding Model: OpenAI, Snowflake, Mistral
	•	Vector Database: Pinecone, Weaviate, MongoDB, Qdrant
	•	Orchestration: LangChain, LlamaIndex, Haystack
	•	Framework: Google Vertex AI, Amazon Bedrock, Microsoft Azure, 
		Oracle OCI, Databricks, Datastax, Snowflake
	•	No/Low Code: LangFlow, Fixie, Verta, Vectara, OpenAI GPTs
	•	Evaluation: RAGas, LangSmith, Arize Phoenix, Quotient

Some, like retrievers, can be selected based on the purpose of your application.  For example, LangChain provides a set of retrievers: Multi-Query Retriever.  Once it’s working, when can we go to the market? How does it get deployed and monitored?  What Responsible AI policies are necessary for your business?  Emerging evaluation, monitoring and observability tools are helping, but we should not rush to market unless the product will succeed. There is always going to balancing act between 1) Accuracy, 2) Performance 3) Latency

Advanced RAG Variants
Agentic RAG: Introduces a loop where the application can use tools and iterate retrieval until the necessary inference is provided. Suitable for complex, multi-step processes.

Streaming RAG: Designed for dynamic data needs, such as providing real-time market updates. Uses tools like Bytewax with Kafka to provide continuous data streams to the vector database.

AI Agents: Beyond information retrieval, AI agents can execute specific tasks autonomously. Applications can be created with multiple agents operating together, executing various tasks, including those using RAG.

AI Product Evaluation Tool - AI App
Currently, the development of Retrieval-Augmented Generation (RAG) applications and agents can appear opaque and challenging for product managers and business development professionals. The complexity and ambition inherent in AI application development often present significant hurdles.
To address these challenges, we propose the creation of AI App, an advanced AI Agent Application designed to streamline and clarify the development process. AI App would generate comprehensive Product Requirements Documents (PRDs) and provide engineers with actionable recommendations, accurate delivery estimates, and insights into partner relations. Additionally, it would identify potential risks and contingencies.

This tool represents a significant opportunity to enhance the planning and execution of RAG applications. By leveraging data-backed planning tools, AI App can ensure accurate estimations and provide clear guidance throughout the development lifecycle, ultimately facilitating more effective and efficient project management.


Conclusion
RAG is a powerful approach that, while not a silver bullet, significantly enhances the relevance and accuracy of AI-generated responses. The field is evolving with various configurations and tools that cater to different use cases. Understanding and utilizing these components effectively can drive the successful deployment of AI applications in the market.

"""

In [ ]:
# existing tools
existing_tools = {
    {
        "name": "LangChain",
        "description": "LangChain is a framework designed to simplify the creation of applications using large language models. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.",
        "link": "https://www.langchain.com/"
    },
    {
        "name": "LlamaIndex",
        "description": "LlamaIndex is a simple, flexible data framework for connecting custom data sources to large language models (LLMs).",
        "link": "https://www.llamaindex.ai/"
    },
    {
        "name": "None",
        "description": "Catch all for non-technical, out-of-scope or infeasible project ideas.",
        "link": None
    },
}

In [ ]:
# existing techniques
existing_techniques = {
    {
        "name": "RAG",
        "description": "RAG is a powerful approach that, while not a silver bullet, significantly enhances the relevance and accuracy of AI-generated responses. The field is evolving with various configurations and tools that cater to different use cases. Understanding and utilizing these components effectively can drive the successful deployment of AI applications in the market.",
        "instruction": RAG
    },
    {
        "name": "Yolo",
        "description": "Just sent it!",
        "instruction": None
    }
}

#### search and match 🔍📇

In [ ]:
# define category
sample_feasible_category = "i have a dataset of jobs and candidates to match"
sample_infeasible_category = "i have a jar of chocolate chip cookies"

# find most related workflow solution
def most_related_workflow(category):
    pass

### Project Proposal Hypothesis

1. 🧠 User Idea
2. Business Opportunity w/ Scope
3. Business Opportunity w/ Scope & ✅

⬇️

Hypothesis:

- What the project is aiming to solve.
- Why this is important.
- Related solution implementation.
- How it will implement:
    - metrics to measure and monitor
    - costs, time, other resources
    - best practices for this implementation
    - how this solution incorporates original idea's specifications

### LLM #4 - Code Implementation 🤖🧠

In [ ]:
def code_llm(proposal):
    pass

### Suggested Solution w/ Explanation 🌟

e.g. code to try